In [1]:
from abc import ABC, abstractmethod

class MnistClassifierInterface(ABC):
    @abstractmethod
    def train(self, X_train, y_train):
        pass

    @abstractmethod
    def predict(self, X_test):
        pass


In [2]:
from sklearn.ensemble import RandomForestClassifier

class RandomForestModel(MnistClassifierInterface):
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)


In [3]:
from tensorflow import keras
from tensorflow.keras import layers

class FFNNModel(MnistClassifierInterface):
    def __init__(self):
        self.model = keras.Sequential([
            layers.Input(shape=(784,)),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(10, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

    def predict(self, X_test):
        return self.model.predict(X_test).argmax(axis=1)


In [4]:
class CNNModel(MnistClassifierInterface):
    def __init__(self):
        self.model = keras.Sequential([
            layers.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(10, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

    def predict(self, X_test):
        return self.model.predict(X_test).argmax(axis=1)


In [5]:
class MnistClassifier:
    def __init__(self, algorithm):
        if algorithm == 'rf':
            self.model = RandomForestModel()
        elif algorithm == 'nn':
            self.model = FFNNModel()
        elif algorithm == 'cnn':
            self.model = CNNModel()
        else:
            raise ValueError("Invalid algorithm. Choose from 'rf', 'nn', or 'cnn'.")

    def train(self, X_train, y_train):
        self.model.train(X_train, y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)


In [6]:
import numpy as np
from tensorflow.keras.datasets import mnist

# Load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

# Train & test Random Forest
clf = MnistClassifier(algorithm='rf')
clf.train(X_train.reshape(-1, 784), y_train)  # RF needs flattened images
predictions = clf.predict(X_test.reshape(-1, 784))
print(f"Random Forest Predictions: {predictions[:10]}")

# Train & test FFNN
clf = MnistClassifier(algorithm='nn')
clf.train(X_train.reshape(-1, 784), keras.utils.to_categorical(y_train, 10))  # One-hot encoding
predictions = clf.predict(X_test.reshape(-1, 784))
print(f"FFNN Predictions: {predictions[:10]}")

# Train & test CNN
clf = MnistClassifier(algorithm='cnn')
clf.train(X_train.reshape(-1, 28, 28, 1), keras.utils.to_categorical(y_train, 10))  # Reshaped images
predictions = clf.predict(X_test.reshape(-1, 28, 28, 1))
print(f"CNN Predictions: {predictions[:10]}")


Random Forest Predictions: [7 2 1 0 4 1 4 9 5 9]
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7916 - loss: 0.6836 - val_accuracy: 0.9638 - val_loss: 0.1298
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9410 - loss: 0.1980 - val_accuracy: 0.9745 - val_loss: 0.0924
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9579 - loss: 0.1430 - val_accuracy: 0.9748 - val_loss: 0.0853
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9642 - loss: 0.1143 - val_accuracy: 0.9765 - val_loss: 0.0768
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9701 - loss: 0.0983 - val_accuracy: 0.9765 - val_loss: 0.0727
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
FFNN Predictions: [7 2 1 0 4 1 4 9 5 9]
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8458 - loss: 0.4906 - val_accuracy: 0.9850 - val_loss: 0.0540
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9767 - loss: 0.0753 - val_accuracy: 0.

In [9]:
### Testing MnistClassifierInterface and Models

In [10]:
issubclass(RandomForestModel, MnistClassifierInterface)


True

In [11]:
issubclass(FFNNModel, MnistClassifierInterface)


True

In [12]:
issubclass(CNNModel, MnistClassifierInterface)

True

In [13]:
## Test Random Forest model

In [14]:
rf_model = RandomForestModel()
#rf_model.train(X_train.reshape(-1, 784), y_train.argmax(axis=1))  # RF needs non-one-hot labels
rf_model.train(X_train.reshape(-1, 784), y_train)  # No need for .argmax(axis=1)

predictions = rf_model.predict(X_test.reshape(-1, 784))
print(f"RF Predictions: {predictions[:10]}")


RF Predictions: [7 2 1 0 4 1 4 9 5 9]


In [15]:
## Test FFNN model

In [16]:
ffnn_model = FFNNModel()
ffnn_model.train(X_train.reshape(-1, 784), keras.utils.to_categorical(y_train, 10))  # FFNN needs one-hot labels
predictions = ffnn_model.predict(X_test.reshape(-1, 784))
print(f"FFNN Predictions: {predictions[:10]}")


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7802 - loss: 0.6991 - val_accuracy: 0.9615 - val_loss: 0.1312
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9432 - loss: 0.1948 - val_accuracy: 0.9733 - val_loss: 0.0925
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9563 - loss: 0.1428 - val_accuracy: 0.9757 - val_loss: 0.0874
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9646 - loss: 0.1175 - val_accuracy: 0.9783 - val_loss: 0.0741
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9701 - loss: 0.0979 - val_accuracy: 0.9780 - val_loss: 0.0735
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
FFNN Predictions: [7 2 1 0 4 1 4 9 5 9]


In [17]:
## test cnn model

In [18]:
cnn_model = CNNModel()
cnn_model.train(X_train.reshape(-1, 28, 28, 1), keras.utils.to_categorical(y_train, 10))  # CNN needs 28x28x1
predictions = cnn_model.predict(X_test.reshape(-1, 28, 28, 1))
print(f"CNN Predictions: {predictions[:10]}")


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8374 - loss: 0.5019 - val_accuracy: 0.9813 - val_loss: 0.0572
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9744 - loss: 0.0826 - val_accuracy: 0.9883 - val_loss: 0.0387
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9829 - loss: 0.0556 - val_accuracy: 0.9897 - val_loss: 0.0346
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9871 - loss: 0.0419 - val_accuracy: 0.9902 - val_loss: 0.0343
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9891 - loss: 0.0358 - val_accuracy: 0.9895 - val_loss: 0.0359
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
CNN Predictions: [7 2 1 0 4 1 4 9 5 9]


In [19]:
## Test the MnistClassifier Wrapper

In [20]:
clf = MnistClassifier(algorithm='rf')
clf.train(X_train.reshape(-1, 784), y_train)
predictions = clf.predict(X_test.reshape(-1, 784))
print(f"MnistClassifier RF Predictions: {predictions[:10]}")


MnistClassifier RF Predictions: [7 2 1 0 4 1 4 9 5 9]


In [21]:
clf = MnistClassifier(algorithm='nn')
clf.train(X_train.reshape(-1, 784), keras.utils.to_categorical(y_train, 10))
predictions = clf.predict(X_test.reshape(-1, 784))
print(f"MnistClassifier FFNN Predictions: {predictions[:10]}")


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7855 - loss: 0.6829 - val_accuracy: 0.9648 - val_loss: 0.1312
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9419 - loss: 0.1948 - val_accuracy: 0.9703 - val_loss: 0.1016
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9575 - loss: 0.1380 - val_accuracy: 0.9743 - val_loss: 0.0856
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9647 - loss: 0.1155 - val_accuracy: 0.9773 - val_loss: 0.0749
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9691 - loss: 0.0987 - val_accuracy: 0.9785 - val_loss: 0.0774
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  
MnistClassifier FFNN Predictions: [7 2 1 0 4 1 4 9 5 9]


In [22]:
clf = MnistClassifier(algorithm='cnn')
clf.train(X_train.reshape(-1, 28, 28, 1), keras.utils.to_categorical(y_train, 10))
predictions = clf.predict(X_test.reshape(-1, 28, 28, 1))
print(f"MnistClassifier CNN Predictions: {predictions[:10]}")


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8429 - loss: 0.5022 - val_accuracy: 0.9813 - val_loss: 0.0586
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9734 - loss: 0.0831 - val_accuracy: 0.9878 - val_loss: 0.0409
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9819 - loss: 0.0579 - val_accuracy: 0.9877 - val_loss: 0.0378
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9858 - loss: 0.0446 - val_accuracy: 0.9908 - val_loss: 0.0337
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9900 - loss: 0.0326 - val_accuracy: 0.9913 - val_loss: 0.0325
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step  
MnistClassifier CNN Predictions: [7 2 1 0 4 1 4 9 5 9]


In [23]:
## Edge Case Tests

In [ ]:
import numpy as np

# Test 1: Passing invalid data type (string instead of image data)
try:
    clf = MnistClassifier(algorithm='rf')
    clf.train("invalid_data", "invalid_labels")  # Should raise an error
except Exception as e:
    print(f"🛑 Error Caught (Invalid Data Type): {e}")

# Test 2: Passing empty data
try:
    clf = MnistClassifier(algorithm='nn')
    clf.train(np.array([]), np.array([]))  # Should raise an error
except Exception as e:
    print(f"🛑 Error Caught (Empty Data): {e}")

# Test 3: Wrong shape for CNN
try:
    clf = MnistClassifier(algorithm='cnn')
    clf.train(np.random.rand(10, 10), np.random.rand(10, 10))  # Should raise an error
except Exception as e:
    print(f"🛑 Error Caught (Wrong Shape): {e}")

# Test 4: Training with NaN values
try:
    clf = MnistClassifier(algorithm='rf')
    X_train_nan = X_train.reshape(-1, 784).copy()
    X_train_nan[0, 0] = np.nan  # Inject NaN into the first sample
    clf.train(X_train_nan, y_train)
except Exception as e:
    print(f"🛑 Error Caught (NaN Values in Input): {e}")

# Test 5: Training with incorrect labels (strings instead of numbers)
try:
    clf = MnistClassifier(algorithm='nn')
    y_train_invalid = np.array(["A"] * len(y_train))  # Non-numeric labels
    clf.train(X_train.reshape(-1, 784), y_train_invalid)
except Exception as e:
    print(f"🛑 Error Caught (Invalid Labels): {e}")

# Test 6: Training with batch size of 1 (Minimal Input)
try:
    clf = MnistClassifier(algorithm='cnn')
    X_train_small = X_train[:1].reshape(1, 28, 28, 1)  # Only one sample
    y_train_small = keras.utils.to_categorical(y_train[:1], 10)
    clf.train(X_train_small, y_train_small)
    print("✅ Passed: Model can train with a batch of 1 sample")
except Exception as e:
    print(f"🛑 Error Caught (Batch Size 1): {e}")


🛑 Error Caught (Invalid Data Type): could not convert string to float: 'invalid_data'
🛑 Error Caught (Empty Data): Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.1`. Either provide more data, or a different value for the `validation_split` argument.
Epoch 1/5
🛑 Error Caught (Wrong Shape): Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", shape=(None, 10), dtype=float32). Expected shape (None, 28, 28, 1), but input has incompatible shape (None, 10)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 10), dtype=float32)
  • training=True
  • mask=None
